In [1]:
# !pip3 install https://$CDSW_DOMAIN/api/v2/python.tar.gz --quiet
# !pip3 install scikit-learn --quiet

This builds and deploys the Pytorch model from the Ray Tune notebook in CML

In [7]:
import cmlapi
import os
import json
import time

api_key = os.environ["CDSW_APIV2_KEY"]
client = cmlapi.default_client(url="https://"+os.environ["CDSW_DOMAIN"], cml_api_key=api_key)
project = client.get_project(os.environ["CDSW_PROJECT_ID"])

In [6]:
model_body = cmlapi.CreateModelRequest(project_id=project.id, name="Ray AMP Model", description="Pytorch model")
model = client.create_model(model_body, project.id)

In [8]:
model_build_body = cmlapi.CreateModelBuildRequest(project_id=project.id, model_id=model.id, file_path="utils/deploy.py", function_name="predict", kernel="python3", runtime_identifier="docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-standard:2023.12.1-b8")
model_build = client.create_model_build(model_build_body, project.id, model.id)

In [9]:
while model_build.status not in ["built", "build failed"]:
    print("waiting for model to build...")
    time.sleep(15)
    model_build = client.get_model_build(project.id, model.id, model_build.id)
if model_build.status == "build failed":
    print("model build failed, see UI for more information")
    sys.exit(1)
print("model built successfully!")

waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting for model to build...
waiting fo

In [12]:
model_deployment_body = cmlapi.CreateModelDeploymentRequest(project_id=project.id, model_id=model.id, build_id=model_build.id)
model_deployment = client.create_model_deployment(model_deployment_body, project.id, model.id, model_build.id)
while model_deployment.status not in ["stopped", "failed", "deployed"]:
    print("waiting for model to deploy...")
    time.sleep(15)
    model_deployment = client.get_model_deployment(project.id, model.id, model_build.id, model_deployment.id)
if model_deployment.status != "deployed":
    print("model deployment failed, see UI for more information")
    sys.exit(1)
print("model deployed successfully!")

waiting for model to deploy...
model deployed successfully!
